In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/complete_spinetta.csv', sep=';')
df.sample(5)

,disc_number,duration_ms,name,track_number,album,artist,author,lyrics_url,lyrics
103,1,254490,La Herida De París,2,Los Niños Que Escriben En El Cielo,Spinetta Jade,Luis Alberto Spinetta,https://genius.com/Spinetta-jade-la-herida-de-...,Lenta bruma cansada de dar al muelle No veo ...
105,1,345500,Sexo,4,Los Niños Que Escriben En El Cielo,Spinetta Jade,Luis Alberto Spinetta,https://genius.com/Spinetta-jade-sexo-lyrics,"Sexo, amo tu sexo mujer No creo en nada si n..."
5,1,248266,Doscientos Años,6,El Jardin de los Presentes,Invisible,Luis Alberto Spinetta,https://genius.com/Invisible-doscientos-anos-l...,Una palabra Dame una palabra... Una palabr...
265,1,209560,Domo Tu,6,Peluson Of Milk,Luis Alberto Spinetta,Luis Alberto Spinetta,https://genius.com/Luis-alberto-spinetta-domo-...,La arena se volvió hacia el mar En un enorme...
310,1,173000,Símil Bahion,5,Mondo Di Cromo,Luis Alberto Spinetta,Luis Alberto Spinetta,NaN,-


## MARKOV

__Get list of all lyrics of non instrumental songs__

In [7]:
lyrics = df[df['lyrics'] != '-'].lyrics.values

In [8]:
def train_markov_chain(lyrics):
    """
    Args:
      - lyrics: a list of strings, where each string represents
                the lyrics of one song by an artist.
    Returns:
      A dict that maps a tuple of 2 words ("bigram") to a list of
      words that follow that bigram, representing the Markov
      chain trained on the lyrics.
    """

    # Initialize the beginning of our chain.
    chain = {
        (None, "<START>"): []
    }

    for lyric in lyrics:
        # Replace newline characters with our tag.
        lyric_newlines = lyric.replace('\n', ' <N> ')
        # Create a tuple representing the most recent (current) bigram.
        last_2 = (None, "<START>")
        for word in lyric_newlines.split():
            # Add the word as one that follows the current bigram.
            chain[last_2].append(word)
            # Shift the current bigram to account for the newly added word.
            last_2 = (last_2[1], word)
            if last_2 not in chain:
                chain[last_2] = []
        chain[last_2].append("<END>")

    return chain

In [9]:
# Train a Markov Chain over all of Logic's lyrics.
chain = train_markov_chain(lyrics)

In [12]:
import random

def generate_new_lyrics(chain):
    """
    Args:
      - chain: a dict representing the Markov chain,
               such as one generated by generate_new_lyrics()
    Returns:
      A string representing the randomly generated song.
    """

    # a list for storing the generated words
    words = []
    # generate the first word
    word = random.choice(chain[(None, "<START>")])
    words.append(word)

    # Begin with the first bigram in our chain.
    last_2 = (None, "<START>")
    while words[-1] != "<END>":
        # Generate the next word.
        word = random.choice(chain[last_2])
        words.append(word)
        # Shift the current bigram to account for the newly added word.
        last_2 = (last_2[1], words[-1])

    # Join the words together into a string with line breaks.
    lyrics = " ".join(words[:-1])
    return "\n".join(lyrics.split("<N>"))

In [13]:
print(generate_new_lyrics(chain))

Vive Hirohito: -"La calle es la luz Tan sólo estando así contigo Yo veo mi elemento Veo en el mar,es un espejo en una cinta con la noche Que gime el viento Todo dura un instante Bajo el sol Y no sé así porque razón Los barcos vienen y se perdió en la adolescencia Correrás al fin Un dulzón, un temor Un escape hacia el mar.
